In [16]:
import pandas as pd
import numpy as np
import copy
import csv

from tqdm import tqdm
from Bio import SeqIO, SearchIO 

In [17]:
rodeo = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/main_co_occur_rodeo_psi.csv', sep = ",")
rodeo = rodeo.loc[:,['Query','Protein_acc']]
rodeo

,Query,Protein_acc
0,WP_145853129.1,WP_187365872.1
1,WP_145853129.1,WP_187365873.1
2,WP_145853129.1,WP_145853125.1
3,WP_145853129.1,WP_145853126.1
4,WP_145853129.1,WP_145853127.1
...,...,...
11152,WP_015759610.1,WP_015759614.1
11153,WP_015759610.1,WP_015759615.1
11154,WP_015759610.1,WP_015759616.1
11155,WP_015759610.1,WP_015759617.1


In [18]:
file = '/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/parsed_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 10
count = 0
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
        res_domains[res.id] = {}
        for hit in res.hits:
            if hit.id[:4] == 'TIGR':
                    call = hit.description.split(":")[0]
            else:
                    call = hit.id
            for hsp in hit.hsps:
                overlap = False
                if res_domains[res.id].keys(): #not empty
                    names = copy.deepcopy(list(res_domains[res.id].keys()))
                    for name in names: #проверка на перекрытие с раннее добавленными генами 
                        for q_range in res_domains[res.id][name]:
                            if min(hsp.env_end, q_range[1]) - max(hsp.env_start, q_range[0]) > x:
                                overlap = True
                                break
                        if overlap:
                            break
                    if not overlap: 
                        if call not in res_domains[res.id]:
                            res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                            
                        if (hsp.env_start, hsp.env_end) not in res_domains[res.id][call]:
                                        #для случая мультидоменного белка с одинаковыми доменами: 
                                        #один и тот же белок выравнивается на несколько мест
                            res_domains[res.id][call].append((hsp.env_start, hsp.env_end))
                       
                else:
                    res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                    

In [19]:
domains_names = {}
for key, value in res_domains.items():
    domains_names[key] = list(value.keys())

domains_names = pd.DataFrame({'domains': domains_names.values()}, index =res_domains.keys())
domains_names

,domains
WP_011566059.1,[MnhB]
WP_011566060.1,[2a6301s01]
WP_011566061.1,"[mycofact_glyco, Methyltransf_23, Methyltransf..."
WP_011566062.1,[Creatininase]
WP_011566063.1,[actino_HemFlav]
...,...
WP_160900598.1,"[DoxX_3, DoxX]"
WP_160901386.1,[DNA_alkylation]
WP_160900597.1,"[Pyr_redox_2, Reductase_C]"
WP_160900596.1,"[FAM70, DUF3040]"


__Составить таблицу, где для каждого MftB будет записано, какие домены есть в соответствующем геномном регионе__

In [20]:
domains_region = {k:[] for k in rodeo['Query'].unique()}

for query in tqdm(domains_region.keys()):
    temp = rodeo[rodeo['Query'] == query]
    for prot_id in temp['Protein_acc']:
        if prot_id in domains_names.index:
            for domains in domains_names.loc[prot_id]:
                domains_region[query] += domains

                
domains_region

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [00:03<00:00, 179.91it/s]


{'WP_145853129.1': ['STAS_2',
  'TetR_C_1',
  'TetR_N',
  'CSD',
  'pcrA',
  'OsmC',
  'mycofact_glyco',
  'mycofact_rSAM',
  'Mycofactocin_RRE',
  'mycofactocin',
  'mycofact_TetR',
  'Glyco_transf_4',
  'Glyoxalase',
  'mycofact_creat',
  'Clp_N',
  'MelC1'],
 'WP_056154608.1': ['efflux_EmrB',
  'DUF6286',
  'mycofact_glyco',
  'SDR_subfam_2',
  'mycofact_creat',
  'actino_HemFlav',
  'mycofact_rSAM',
  'Mycofactocin_RRE',
  'mycofactocin',
  'mycofact_TetR',
  'Flavin_Reduct',
  'pcaR_pcaU',
  'TetR_N',
  'betaine_BetI',
  'TetR_C_31',
  'TetR_N',
  'betaine_BetI',
  'Peripla_BP_6',
  'BPD_transp_2',
  'urea_trans_UrtD',
  'BPD_transp_2'],
 'WP_111258449.1': ['PemK_toxin',
  'HicB',
  'Arc',
  'WYL',
  'HTH_11',
  'HTH_PafC',
  'EHN',
  'EHN',
  'bchO_mg_che_rel',
  'mycofact_glyco',
  'mycofact_TetR',
  'mycofactocin',
  'Mycofactocin_RRE',
  'mycofact_rSAM',
  'SDR_subfam_1',
  'SDR_subfam_1',
  'SDR_subfam_1',
  'SDR_subfam_1',
  'AAA_5',
  'VWA_2',
  'marine_srt_targ',
  'mycofa

In [21]:
dom_region = pd.DataFrame({'domains': domains_region.values()} , index = domains_region.keys())
dom_region

,domains
WP_145853129.1,"[STAS_2, TetR_C_1, TetR_N, CSD, pcrA, OsmC, my..."
WP_056154608.1,"[efflux_EmrB, DUF6286, mycofact_glyco, SDR_sub..."
WP_111258449.1,"[PemK_toxin, HicB, Arc, WYL, HTH_11, HTH_PafC,..."
WP_212516844.1,"[3oxo_ACP_reduc, kduD, pyruv_carbox, FCD, GntR..."
WP_040718660.1,"[NDMA_methanol, AAA_5, Sigma54_activ_2, 26Sp45..."
...,...
WP_212844265.1,"[UreD, salicylate_mono, TetR_C_31, Laminin_G_3..."
WP_050670291.1,"[2TM, DUF779, Aldedh, actino_HemFlav, mycofact..."
WP_122191250.1,"[Aldedh, PA_CoA_ligase, Rxyl_3153, adh_short_C..."
WP_011208766.1,"[VWA_CoxE, Methyltransf_25, meth_Rta_06860, PA..."


__Добавим столбик с именем ноды MftB в сетке, типа zz00000. И стобики с да/нет для топ встречаемых доменов, возьмем топ 25 из каунтов в геномных регионах__

In [22]:
dict_acc = dict()
with open('mftb_cluster_rep_seq_fasta default node.csv', newline='') as csvfile2:
    file = csv.reader(csvfile2, delimiter=',', quotechar='|')
    for row in file:
        name = row[0].split(" ")[0]
        dict_acc[name] = row[1]
dict_acc.pop('Description')
dict_acc_df = pd.DataFrame({'acc_cyt': dict_acc.values()}, index = dict_acc.keys())
dict_acc_df

,acc_cyt
WP_239101810.1,zzzz657
WP_228638331.1,zzzz574
WP_189138050.1,zzzzz54
WP_255280863.1,zzzz622
WP_081878369.1,zzzz154
...,...
WP_257865691.1,zzzz347
WP_214290027.1,zzzz141
WP_119595456.1,zzzz167
WP_182609472.1,zzzz169


In [23]:
df_temp_region = dict_acc_df.join(dom_region)
df_temp_region = df_temp_region.dropna(subset = 'domains')
df_temp_region

,acc_cyt,domains
WP_228638331.1,zzzz574,"[mycofact_glyco, SDR_subfam_2, mycofact_creat,..."
WP_189138050.1,zzzzz54,"[3a0107s01c2, 3a0107s02c, phosphate_pstC, ptsS..."
WP_081878369.1,zzzz154,"[decarb_PcaC, DJ-1_PfpI, Aldo_ket_red, HTH_3, ..."
WP_209898104.1,zzzz630,"[CxxC_CxxC_SSSS, zf-HIT, zf-ribbon_3, protocat..."
WP_084483987.1,zzzz672,"[TetR_N, mycofact_TetR, SPW, Pyr_redox_2, Redu..."
...,...,...
WP_194624246.1,zzzz338,"[Mycofactocin_RRE, mycofact_rSAM, actino_HemFl..."
WP_214290027.1,zzzz141,"[Glyoxalase_6, DUF2332, mycofact_TetR, mycofac..."
WP_119595456.1,zzzz167,"[mycofact_TetR, mycofactocin, Mycofactocin_RRE]"
WP_182609472.1,zzzz169,"[HpcH_HpaI, drrA, ABC2_membrane, drrC, DUF4442..."


In [24]:
top25_region = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_counts_region_10.csv', sep = ',', index_col = 0, nrows = 25)
dict_add_region = dict()
for name in top25_region.index:
    dict_add_region[name] = [0]
dict_add_region = pd.DataFrame(dict_add_region, index = df_temp_region.index)
df_to_color_region = df_temp_region.join(pd.DataFrame(dict_add_region))


for mftb in df_to_color_region.index:
    for prot in df_to_color_region.columns[2:]:
        if prot in df_to_color_region.loc[(mftb, 'domains')]:
            df_to_color_region.loc[(mftb, prot)] += 1  

df_to_color_region

,acc_cyt,domains,mycofactocin,Mycofactocin_RRE,mycofact_rSAM,mycofact_TetR,actino_HemFlav,mycofact_glyco,mycofact_creat,SDR_subfam_1,...,GerE,mycofact_OYE_2,Response_reg,NDMA_methanol,PA_CoA_Oxy5,HisKA_3,mycofact_OYE_1,Methyltransf_11,efflux_EmrB,Fe-ADH
WP_228638331.1,zzzz574,"[mycofact_glyco, SDR_subfam_2, mycofact_creat,...",1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
WP_189138050.1,zzzzz54,"[3a0107s01c2, 3a0107s02c, phosphate_pstC, ptsS...",1,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
WP_081878369.1,zzzz154,"[decarb_PcaC, DJ-1_PfpI, Aldo_ket_red, HTH_3, ...",1,1,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
WP_209898104.1,zzzz630,"[CxxC_CxxC_SSSS, zf-HIT, zf-ribbon_3, protocat...",1,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
WP_084483987.1,zzzz672,"[TetR_N, mycofact_TetR, SPW, Pyr_redox_2, Redu...",1,1,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_194624246.1,zzzz338,"[Mycofactocin_RRE, mycofact_rSAM, actino_HemFl...",0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
WP_214290027.1,zzzz141,"[Glyoxalase_6, DUF2332, mycofact_TetR, mycofac...",1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_119595456.1,zzzz167,"[mycofact_TetR, mycofactocin, Mycofactocin_RRE]",1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_182609472.1,zzzz169,"[HpcH_HpaI, drrA, ABC2_membrane, drrC, DUF4442...",1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#df_to_color_region.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/domains_per_region.csv')

---

__Составить таблицу, где для каждого MftB будет записано, какие домены есть в соответствующем псевдоопероне__

In [25]:
operon_prot_id = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/operon_100_protIDs_rodeo.csv', sep = ",", index_col = 0)
operon_prot_id

,Query,Protein_acc
1,WP_145853129.1,WP_246115648.1
2,WP_145853129.1,WP_246115687.1
3,WP_145853129.1,WP_145853128.1
4,WP_145853129.1,WP_145853129.1
5,WP_145853129.1,WP_145853130.1
...,...,...
5243,WP_015759610.1,WP_015759614.1
5244,WP_015759610.1,WP_015759615.1
5245,WP_015759610.1,WP_015759616.1
5246,WP_015759610.1,WP_015759617.1


In [26]:
domains_operon = {k:[] for k in operon_prot_id['Query'].unique()}

for query in tqdm(domains_operon.keys()):
    temp = operon_prot_id[operon_prot_id['Query'] == query]
    for prot_id in temp['Protein_acc']:
        if prot_id in domains_names.index:
            for domains in domains_names.loc[prot_id]:
                domains_operon[query] += domains

                
domains_operon

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 675/675 [00:01<00:00, 341.74it/s]


{'WP_145853129.1': ['OsmC',
  'mycofact_glyco',
  'mycofact_rSAM',
  'Mycofactocin_RRE',
  'mycofactocin',
  'Clp_N'],
 'WP_056154608.1': ['actino_HemFlav',
  'mycofact_rSAM',
  'Mycofactocin_RRE',
  'mycofactocin',
  'Flavin_Reduct',
  'pcaR_pcaU',
  'TetR_N',
  'betaine_BetI',
  'TetR_C_31',
  'TetR_N',
  'betaine_BetI'],
 'WP_111258449.1': ['mycofactocin',
  'Mycofactocin_RRE',
  'mycofact_rSAM',
  'SDR_subfam_1',
  'SDR_subfam_1',
  'SDR_subfam_1',
  'SDR_subfam_1',
  'AAA_5',
  'VWA_2',
  'marine_srt_targ',
  'mycofact_creat'],
 'WP_212516844.1': ['FCD',
  'GntR',
  'P450_rel_GT_act',
  'mycofactocin',
  'Mycofactocin_RRE',
  'mycofact_rSAM',
  'actino_HemFlav',
  'mycofact_creat',
  'mycofact_glyco'],
 'WP_040718660.1': ['actino_HemFlav',
  'mycofact_rSAM',
  'Mycofactocin_RRE',
  'mycofactocin',
  'Cupin_6',
  'HTH_18',
  'DUF664'],
 'WP_188782448.1': ['mycofact_creat',
  'actino_HemFlav',
  'mycofact_rSAM',
  'Mycofactocin_RRE',
  'mycofactocin',
  'Response_reg',
  'GerE',
  '

__Возьмем топ 25 из каунтов в псевдооперонах__

In [27]:
dom_operon = pd.DataFrame({'domains': domains_operon.values()} , index = domains_operon.keys())

df_temp_operon = dict_acc_df.join(dom_operon)
df_temp_operon = df_temp_operon.dropna(subset = 'domains')

top25_operon = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_counts_operon_100.csv', sep = ',', index_col = 0, nrows = 25)
dict_add_operon = dict()
for name in top25_operon.index:
    dict_add_operon[name] = [0]
dict_add_operon = pd.DataFrame(dict_add_operon, index = df_temp_operon.index)
df_to_color_operon = df_temp_operon.join(pd.DataFrame(dict_add_operon))


for mftb in df_to_color_operon.index:
    for prot in df_to_color_operon.columns[2:]:
        if prot in df_to_color_operon.loc[(mftb, 'domains')]:
            df_to_color_operon.loc[(mftb, prot)] += 1  

df_to_color_operon

,acc_cyt,domains,Mycofactocin_RRE,mycofact_rSAM,mycofactocin,actino_HemFlav,mycofact_creat,mycofact_glyco,SDR_subfam_1,SDR_subfam_2,...,TetR_N,ETF_alpha,HisKA_3,DNA_alkylation,ETF,Fe-ADH,GG-red-SF,AAA_5,mycofact_TetR,HTH_AsnC-type
WP_228638331.1,zzzz574,"[mycofact_creat, mycofact_rSAM, mycofact_MftB,...",0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_189138050.1,zzzzz54,"[mycofact_rSAM, rad_SAM_trio, actino_HemFlav, ...",0,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
WP_081878369.1,zzzz154,"[mycofact_glyco, mycofact_rSAM, Mycofactocin_R...",1,1,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
WP_209898104.1,zzzz630,"[mycofact_rSAM, Mycofactocin_RRE, AA-adenyl-do...",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_084483987.1,zzzz672,"[DNA_alkylation, mycofactocin, mycofact_rSAM, ...",1,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_194624246.1,zzzz338,"[Mycofactocin_RRE, mycofact_rSAM]",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_214290027.1,zzzz141,"[mycofactocin, Mycofactocin_RRE]",1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_119595456.1,zzzz167,"[mycofactocin, Mycofactocin_RRE]",1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
WP_182609472.1,zzzz169,"[mycofactocin, Mycofactocin_RRE]",1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
#df_to_color_operon.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/domains_per_operon.csv')